In [7]:
import json
import os

from azureml.core import Workspace
from azureml.core.model import Model
from dotenv import load_dotenv

load_dotenv()

SUBSCRIPTION_ID = os.getenv("SUBSCRIPTION_ID")

if not SUBSCRIPTION_ID:
    raise ValueError(
        "Environment variable SUBSCRIPTION_ID is not set."
        " Please set it before running the script."
    )

RESOURCE_GROUP = "idm-dev-azureml-1"
LOCATION = "eastus"
MODEL_NAME = "model"

In [8]:
workspace = Workspace.create(
    name="ws-hw-azureml-2",
    subscription_id=SUBSCRIPTION_ID,
    resource_group=RESOURCE_GROUP,
    location=LOCATION,
)

registered_model = Model.register(
    model_path="model.pkl",
    model_name=MODEL_NAME,
    workspace=workspace,
)

registered_model

Deploying AppInsights with name wshwazurinsightsb60fc619.
Deployed AppInsights with name wshwazurinsightsb60fc619. Took 2.73 seconds.
Deploying KeyVault with name wshwazurkeyvaultb5bbf515.
Deploying StorageAccount with name wshwazurstorage49cd5deb0.
Deployed KeyVault with name wshwazurkeyvaultb5bbf515. Took 18.07 seconds.
Deploying Workspace with name ws-hw-azureml-2.
Deployed StorageAccount with name wshwazurstorage49cd5deb0. Took 22.1 seconds.
Deployed Workspace with name ws-hw-azureml-2. Took 21.41 seconds.
Registering model model


Model(workspace=Workspace.create(name='ws-hw-azureml-2', subscription_id='25847311-05cf-407c-b6b9-6ba67c170430', resource_group='idm-dev-azureml-1'), name=model, id=model:1, version=1, tags={}, properties={})

In [7]:
import json
import joblib
import numpy as np
import pandas as pd
# from azureml.core.model import Model


def init():
    global model

    # model_path = Model.get_model_path(f"{MODEL_NAME}.pkl")
    model = joblib.load(f"{MODEL_NAME}.pkl")


def run(raw_data):
    try:  ## Try la predicción.
        data = json.loads(raw_data)["data"][0]
        data = pd.DataFrame(data)

        columns_model = [
            " Borrowing dependency",
            " Current Liability to Current Assets",
            " Net Value Per Share (C)",
            " Non-industry income and expenditure/revenue",
            " Net Value Growth Rate",
            " Continuous interest rate (after tax)",
        ]

        data_dummies = data[columns_model]
        result = model.predict(data_dummies).tolist()

        return json.dumps(result)
    except Exception as e:
        print("Error in prediction:", str(e))
        return json.dumps(str(e))


init()

result = run(
    json.dumps(
        {
            "data": [
                {
                    " Borrowing dependency": [10000000],
                    " Current Liability to Current Assets": [10000000],
                    " Net Value Per Share (C)": [1000],
                    " Non-industry income and expenditure/revenue": [10000000],
                    " Net Value Growth Rate": [10000000],
                    " Continuous interest rate (after tax)": [10000000],
                }
            ]
        }
    )
)

result

'[0]'

In [9]:
################ score.py ###################


scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{MODEL_NAME}')
  model = joblib.load(model_path)

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    columns_model = [
      " Borrowing dependency",
      " Current Liability to Current Assets",                       
      " Net Value Per Share (C)",                                  
      " Non-industry income and expenditure/revenue",              
      " Net Value Growth Rate",                                    
      " Continuous interest rate (after tax)",
    ]

    data_dummies = data[columns_model]
    result = model.predict(data_dummies).tolist()

    return json.dumps(result)
  except Exception as e:
    return json.dumps(str(e))
"""

with open("score.py", "w") as file:
    file.write(scorepy)

In [10]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

venv = Environment("env-hw-2")

## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!
venv.python.conda_dependencies = CondaDependencies.create(
    conda_packages=[
        "pandas",
        "scikit-learn",
    ]
)

inference_config = InferenceConfig(
    environment=venv,
    entry_script="score.py",
)

## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=4,
)

service = Model.deploy(
    workspace=workspace,
    name="service-2",
    models=[registered_model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True,
)

/var/folders/y2/d02sw9ns6r772cc_ln_3rgym0000gn/T/ipykernel_5211/106412736.py:29: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


In [ ]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")

file.write(scoreuri)
file.close()


In [6]:
print(service.get_logs())

None
